In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
url ="https://raw.githubusercontent.com/Sooryak12/FakeReviewDetectionLSTM/master/selected.txt"

In [12]:
df=pd.read_csv(url,delimiter="\t")

In [13]:
df["LABEL"] = df["LABEL"].replace(['__label2__', '__label1__'], [0, 1])
df["VERIFIED_PURCHASE"] = df["VERIFIED_PURCHASE"].replace(['N', 'Y'], [0, 1])


In [14]:
df.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,1,4,0,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,1,4,1,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,1,3,0,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,1,4,0,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,1,4,0,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [15]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df[["REVIEW_TEXT", "RATING", "VERIFIED_PURCHASE"]], df["LABEL"], test_size=0.2)

In [16]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Trained_Tfidf_vect = Tfidf_vect.fit(Train_X["REVIEW_TEXT"].values)

Train_X_Tfidf = Trained_Tfidf_vect.transform(Train_X["REVIEW_TEXT"].values)
Test_X_Tfidf = Trained_Tfidf_vect.transform(Test_X["REVIEW_TEXT"].values)

In [18]:
from sklearn import  svm
from sklearn.metrics import accuracy_score

In [19]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf, Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score:", accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score: 64.76190476190476


In [20]:
from scipy.sparse import  hstack

In [21]:
import pickle

In [22]:
print (Train_X_Tfidf.shape, '\n')

(16800, 5000) 



In [23]:
Train_VP_Column = [ [vp] for vp in Train_X["VERIFIED_PURCHASE"].values ]
print ("sample vp column:", Train_VP_Column[:4])

Train_VP_Column = np.array(Train_VP_Column)
print (Train_VP_Column.shape)

sample vp column: [[0], [1], [1], [0]]
(16800, 1)


In [24]:
New_Train_X_Tfidf = hstack((Train_X_Tfidf, Train_VP_Column))  # adding "VERIFIED_PURCHASE" to the Input
print (New_Train_X_Tfidf.shape)

(16800, 5001)


In [28]:
Test_VP_Column = [ [vp] for vp in Test_X["VERIFIED_PURCHASE"].values ] 

In [30]:
New_Test_X_Tfidf = hstack((Test_X_Tfidf, Test_VP_Column))

In [33]:
svmmodel= svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svmmodel.fit(New_Train_X_Tfidf, Train_Y)

predictions_SVM = svmmodel.predict(New_Test_X_Tfidf)
print("SVM Accuracy Score:", accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score: 80.69047619047619


In [34]:
# save the model to disk
filename = 'svmmodelup.sav'
pickle.dump(svmmodel, open(filename, 'wb'))